In [46]:
import os

import numpy as np
import spacy
import torch
from datasets import load_dataset, Dataset, concatenate_datasets
from torch import nn
from torch.nn import functional as F

In [50]:
from torch.utils.data import DataLoader
from util import BucketBatchSampler, BucketDataset

nlp = spacy.load('en_core_web_sm')
dataset = load_dataset('Sp1786/multiclass-sentiment-analysis-dataset')
train_dataset: Dataset = dataset['train']
validation_dataset: Dataset = dataset['validation']
test_dataset: Dataset = dataset['test']

corpus = concatenate_datasets([train_dataset, validation_dataset])['text']
vocabulary = sorted(set(''.join(corpus)))

char_to_i = {u: i for i, u in enumerate(vocabulary)}


def encode_x(char_to_i, message):
    return np.array([char_to_i[char] for char in message])


def encode_y(label):
    vector = torch.zeros(3)
    vector[label] = 1
    return vector


train_messages = [encode_x(char_to_i, message) for message in train_dataset['text']]
train_labels = [encode_y(label) for label in train_dataset['label']]

train_bucket_batch_sampler = BucketBatchSampler(train_messages, 128)  # <-- does not store X
train_bucket_dataset = BucketDataset(train_messages, train_labels)
train_dataloader = DataLoader(train_bucket_dataset, batch_size=1, batch_sampler=train_bucket_batch_sampler,
                              shuffle=False, num_workers=8, drop_last=False)


In [54]:
# See https://docs.pytorch.org/docs/stable/generated/torch.nn.GRU.html.
class Model(nn.Module):

    def __init__(self, vocabulary_size, embedding_dim, hidden_size):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocabulary_size, embedding_dim=embedding_dim)
        self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 3)

    def forward(self, sequence, hidden):
        # (batch_size, sequence_length) -> (batch_size, sequence_length, embedding_dim)
        embedded = self.embedding(sequence)
        # (batch_size, sequence_length, embedding_dim)
        # -> (batch_size, sequence_length, hidden_size), (num_layers, batch_size, hidden_size)
        prediction, hidden = self.rnn(embedded, hidden)
        # See https://docs.pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html#creating-the-network.
        return self.linear(hidden[0]), hidden

In [55]:
embedding_dim = 256
hidden_size = 1024
num_layers = 3
model = Model(len(vocabulary), embedding_dim, hidden_size)
loss_fn = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

# See https://docs.pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop.
for epoch, data in enumerate(train_dataloader):
    x, y = data
    batch_size = x.shape[0]
    h0 = torch.zeros(num_layers, batch_size, hidden_size)
    # See https://discuss.pytorch.org/t/gru-and-padded-sequences-tipps-and-tricks/90729.
    prediction, _ = model.forward(x, h0)
    print('prediction.shape', prediction.shape)
    print('y.shape', y.shape)

    loss = loss_fn(prediction, y)
    loss.backward()
    print(f'Epoch {epoch}, loss {loss.item()}')

    optimizer.step()
    optimizer.zero_grad()
    break

prediction.shape torch.Size([4, 3])
y.shape torch.Size([4, 3])
Epoch 0, loss 1.1266924142837524


In [77]:
train_xs = torch.stack([encode_x(vocabulary, tokens) for tokens in train_message_to_tokens])
train_ys = torch.stack([encode_y(label) for label in train_dataset['label']])
validation_xs = torch.stack([encode_x(vocabulary, tokens) for tokens in validation_message_to_tokens])
validation_ys = torch.stack([encode_y(label) for label in validation_dataset['label']])
print(train_xs.shape)
print(train_ys.shape)
print(validation_xs.shape)
print(validation_ys.shape)

torch.Size([31232, 31839])
torch.Size([31232, 3])
torch.Size([5205, 31839])
torch.Size([5205, 3])


In [78]:
def create_batch(xs, ys, batch_size):
    random_indices = np.random.choice(len(xs), batch_size)
    return xs[random_indices], ys[random_indices]

In [79]:
def determine_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'


device = determine_device()
print(f'Device is {device}')

Device is mps


In [83]:
def estimate_loss(model, iterations, validation_xs, validation_ys, batch_size):
    model.eval()
    loses = torch.zeros(iterations)
    for i in range(iterations):
        validation_x_batch, validation_y_batch = create_batch(validation_xs, validation_ys, batch_size)
        validation_prediction = model(validation_x_batch.to(device))
        validation_loss = F.cross_entropy(validation_prediction, validation_y_batch.to(device), reduction='mean')
        loses[i] = validation_loss.item()
    model.train()
    return loses.mean()

In [84]:
# nn.Softmax is not required (see the nn.CrossEntropyLoss docs why) and can be kept only in the inference
model = nn.Sequential(
    nn.Linear(len(vocabulary), 256),
    nn.Linear(256, 3)
).to(device)

In [85]:
model_dir = os.path.join(os.getcwd(), "models")
os.makedirs(model_dir, exist_ok=True)

batch_size = 512
loss_fn = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

number_of_epoches = 100
min_validation_loss = float('inf')
for epoch in range(number_of_epoches):
    x_batch, y_batch = create_batch(train_xs, train_ys, batch_size)
    prediction = model(x_batch.to(device))
    loss = loss_fn(prediction, y_batch.to(device))

    loss.backward()
    optimizer.step()
    model.zero_grad()

    if epoch % 10 == 0 or epoch == number_of_epoches - 1:
        mean_loss = estimate_loss(model, 10, validation_xs, validation_ys, batch_size)
        print(f'Epoch {epoch}, trail loss {loss.item()}, validation loss {mean_loss.item()}')
        if mean_loss < min_validation_loss:
            model_file_name = os.path.join(model_dir, "model_" + str(epoch) + ".pt")
            torch.save(model.state_dict(), model_file_name)
            print("Model has been saved as", model_file_name)
            min_validation_loss = mean_loss

Epoch 0, trail loss 1.102682113647461, validation loss 1.077303171157837
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_0.pt
Epoch 10, trail loss 0.8979932069778442, validation loss 0.9276639819145203
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_10.pt
Epoch 20, trail loss 0.7764990329742432, validation loss 0.8771727681159973
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_20.pt
Epoch 30, trail loss 0.6884598135948181, validation loss 0.8671800494194031
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_30.pt
Epoch 40, trail loss 0.7256267070770264, validation loss 0.9020015597343445
Epoch 50, trail loss 0.6767445802688599, validation loss 0.8644858598709106
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_50.pt
Epoch 60, trail loss 0.6512887477

In [86]:
model.load_state_dict(
    torch.load(os.path.join(os.getcwd(), "models", 'model_50.pt'), map_location=torch.device(device)))

<All keys matched successfully>

In [91]:
model.eval()
message = "You're the best"
test_tokens, _ = tokenize([message])
x = encode_x(vocabulary, test_tokens)
y = model(x.to(device))

distribution = torch.nn.functional.softmax(y, dim=-1)
print(distribution)
answer = torch.argmax(distribution)
print(['negative', 'neutral', 'positive'][answer])
model.train()

tensor([0.0780, 0.2832, 0.6388], device='mps:0', grad_fn=<SoftmaxBackward0>)
positive


Sequential(
  (0): Linear(in_features=31839, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=3, bias=True)
)

In [30]:
torch.onnx.export(model, (torch.tensor(x, dtype=torch.float32).to(device),), "model.onnx")